In [2]:
# Creating your own sequences

class Items:
    def __init__(self, *values):
        self._values = list(values)
    def __len__(self):
        return len(self._values)
    def __getitem__(self, item):
        return self._values.__getitem__(item)
    def __setitem__(self, key, value):
        self._values.__setitem__(key, value)
    def __str__(self) -> str:
        return str(self._values)

my_items = Items(1, 2, 3)

print(len(my_items))
print(str(my_items))
print(my_items[1])
for item in my_items:
    print(item)
my_items[2] = 4
print(my_items)



3
[1, 2, 3]
2
1
2
3
[1, 2, 4]


In [3]:
# Context managers

def stop_database():
    print("systemctl stop postgresql.service")
def start_database():
    print("systemctl start postgresql.service")

class DBHandler:
    def __enter__(self):
        stop_database()
        return self

    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()

def db_backup():
    print("pg_dump database")

with DBHandler():
    db_backup()


import contextlib

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()

with db_handler():
    db_backup()


systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service
systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [4]:
# Underscores in Python

class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60
        self.__timeout = 120

    def connect(self):
        print("Connecting to {} with timeout {}".format(self.source, self.__timeout))

conn = Connector("postgresql://localhost")
print(conn.source)
print(conn._timeout)
print(conn.__dict__)
print(vars(conn))
print(conn._Connector__timeout)
conn._Connector__timeout = 30
conn.connect()
# print(conn.__timeout) AttributeError: 'Connector' object has no attribute '__timeout'

postgresql://localhost
60
{'source': 'postgresql://localhost', '_timeout': 60, '_Connector__timeout': 120}
{'source': 'postgresql://localhost', '_timeout': 60, '_Connector__timeout': 120}
120
Connecting to postgresql://localhost with timeout 30


In [5]:
# Properties

import re
EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+\.[^@]+")
def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None
class User:
    def __init__(self, username):
        self.username = username
        self._email = None
    @property
    def email(self):
        return self._email
    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f"Can't set {new_email} as it's not a valid email")
        self._email = new_email


# u1 = User("jsmith")
# u1.email = "jsmith@" # ValueError: Can't set jsmith@ as it's not a valid email





In [6]:
# Creating iterable objects

from datetime import timedelta, date
class DateRangeIterable:
    """An iterable that contains its own iterator object."""
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date
    def __iter__(self):
        return self
    def __next__(self):
        if self._present_day >= self.end_date:
            raise StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today

# for day in DateRangeIterable(date(2018, 1, 1), date(2018, 1, 5)):
#     print(day)

# r = DateRangeIterable(date(2018, 1, 1), date(2018, 1, 5))
# print(next(r))
# print(next(r))
# print(next(r))
# print(next(r))
# print(next(r)) # StopIteration

class DateRangeContainerIterable:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days=1)

r1 = DateRangeContainerIterable(date(2018, 1, 1), date(2018, 1, 5))
print(", ".join(map(str, r1)))

print(max(r1))


2018-01-01, 2018-01-02, 2018-01-03, 2018-01-04
2018-01-04


In [7]:
# Creating sequences
class DateRangeSequence:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._range = self._create_range()
    def _create_range(self):
        days = []
        current_day = self.start_date
        while current_day < self.end_date:
            days.append(current_day)
            current_day += timedelta(days=1)
        return days
    def __getitem__(self, day_no):
        return self._range[day_no]
    def __len__(self):
        return len(self._range)

s1 = DateRangeSequence(date(2018, 1, 1), date(2018, 1, 5))
for day in s1:
    print(day)

print(s1[0])
print(s1[-1])


2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-01
2018-01-04


In [8]:
# Container objects

# Let's say we have to mark some points on a map of a game that has two-dimensional
# coordinates. We might expect to find a function like the following:

class Coord:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __iter__(self):
        return (self.x, self.y).__iter__()
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    def __hash__(self):
        return hash((self.x, self.y))

class Boundaries:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    def __contains__(self, coord):
        x, y = coord
        return 0 <= x < self.width and 0 <= y < self.height

class Grid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.limits = Boundaries(width, height)
        self._values = [[None] * width for _ in range(height)]
    def __contains__(self, coord):
        return coord in self.limits
    def __len__(self):
        return len(self._values)
    def __getitem__(self, item):
        return self._values.__getitem__(item)
    def __setitem__(self, coord: Coord, value):
        self._values.__getitem__(coord.x).__setitem__(coord.y, value)
    def __str__(self) -> str:
        return str(self._values)


MARKED = "MARKED"

# Using if method
def mark_coordinate(grid, coord):
    if 0 <= coord.x < grid.width and 0 <= coord.y < grid.height:
        grid[coord] = MARKED

# Using __contains__ method
def mark_coordinate_contains(grid, coord):
    if coord in grid:
        grid[coord] = MARKED

grid = Grid(10, 10)

mark_coordinate(grid, Coord(5, 5))
mark_coordinate_contains(grid, Coord(9, 9))
mark_coordinate_contains(grid, Coord(0, 0))
print(grid)

[['MARKED', None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, 'MARKED', None, None, None, None], [None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, 'MARKED']]


In [9]:
# Dynamic attributes for objects

class DynamicAttributes:
    def __init__(self, attribute):
        self.attribute = attribute
    def __getattr__(self, attr):
        if attr.startswith("fallback_"):
            name = attr.replace("fallback_", "")
            return f"[fallback resolved] {name}"
        raise AttributeError(
            f"{self.__class__.__name__} has no attribute {attr}"
        )

# Here are some calls to an object of this class:
# dyn = DynamicAttributes("value")
# print(dyn.attribute)  # 'value'
# print(dyn.fallback_test) # '[fallback resolved] test'
# dyn.__dict__["fallback_new"] = "new value"
# print(dyn.fallback_new) # 'new value'
# print(getattr(dyn, "something", "default")) # 'default'
# print(dyn.att_test) # AttributeError: DynamicAttributes has no attribute att_test

class WithDynamicAttributes:
    def __init__(self, attributes = {}):
        self.attributes = attributes

    def __getattr__(self, attr):
        if attr in self.attributes.keys():
            return self.attributes[attr]
        raise AttributeError(
            f"{self.__class__.__name__} has no attribute {attr}"
        )
    def __setattr__(self, attr, value):
        if attr == "attributes":
            super().__setattr__(attr, value)
            return
        elif attr in self.attributes.keys():
            self.attributes[attr] = value
            return
        raise AttributeError(
            f"{self.__class__.__name__} has no attribute {attr}"
        )

    def __setitem__(self, key, value):
        self.__setattr__(key, value)

    def __getitem__(self, key):
        return self.attributes[key]


dynamicDict = WithDynamicAttributes({"name": "John", "age": 20})
print(dynamicDict.attributes["name"])
print(dynamicDict.name)

dynamicDict.name = "Joe"
print(dynamicDict.name)
print(dynamicDict.attributes["name"])

# print(dynamicDict.length) # AttributeError: WithDynamicAttributes has no attribute length
# dynamicDict.length = 10 # AttributeError: WithDynamicAttributes has no attribute length

dynamicDict = WithDynamicAttributes({"name": "John", "age": 20, "length": 10})
print(dynamicDict.length)
dynamicDict.length = 20 # AttributeError: WithDynamicAttributes has no attribute length
print(dynamicDict.length)

print(dynamicDict.attributes)

print(dynamicDict["name"])
dynamicDict["name"] = "Joe"
print(dynamicDict["name"])
print(dynamicDict.name)
# print(dynamicDict["aa"])
# dynamicDict["que"] = "test" # AttributeError: WithDynamicAttributes has no attribute que


John
John
Joe
Joe
10
20
{'name': 'John', 'age': 20, 'length': 20}
John
Joe
Joe


In [10]:
# A python class that acts like dict

class MyMapping(dict):

    def __setitem__(self, key, item):
        self.__dict__[key] = item

    def __getitem__(self, key):
        return self.__dict__[key]

    def __repr__(self):
        return repr(self.__dict__)

    def __len__(self):
        return len(self.__dict__)

    def __delitem__(self, key):
        del self.__dict__[key]

    def clear(self):
        return self.__dict__.clear()

    def copy(self):
        return self.__dict__.copy()

    def has_key(self, k):
        return k in self.__dict__

    def update(self, *args, **kwargs):
        return self.__dict__.update(*args, **kwargs)

    def keys(self):
        return self.__dict__.keys()

    def values(self):
        return self.__dict__.values()

    def items(self):
        return self.__dict__.items()

    def pop(self, *args):
        return self.__dict__.pop(*args)

    def __cmp__(self, dict_):
        return self.__cmp__(self.__dict__, dict_)

    def __contains__(self, item):
        return item in self.__dict__

    def __iter__(self):
        return iter(self.__dict__)

    def __unicode__(self):
        return repr(self.__dict__)
        # return unicode(repr(self.__dict__))


o = MyMapping()
o.foo = "bar"
o['lumberjack'] = 'foo'
o.update({'a': 'b'}, c=44)
print('lumberjack' in o)
print(o)

# In [187]: run mapping.py
# True
# {'a': 'b', 'lumberjack': 'foo', 'foo': 'bar', 'c': 44}

True
{'foo': 'bar', 'lumberjack': 'foo', 'a': 'b', 'c': 44}


In [11]:
# Callable objects

from collections import defaultdict
class CallCount:
    def __init__(self):
        self._counts = defaultdict(int)
    def __call__(self, argument):
        self._counts[argument] += 1
        return self._counts[argument]


cc = CallCount()
print("cc(1):", cc(1))
print("cc(2):", cc(2))
print("cc(1):", cc(1))
print("cc(1):", cc(1))
print("cc(\"something\"):", cc("something"))

cc(1): 1
cc(2): 1
cc(1): 2
cc(1): 3
cc("something"): 1


## Summary of magic methods
We can summarize the concepts we described in the previous sections in the form of a cheat
sheet like the one presented as follows. For each action in Python, the magic method
involved is presented, along with the concept that it represents:

| Statement | Magic method | Python concept |
| :-- | :-: | :-- |
| `obj[key]`<br/> `obj[i:j]`<br/> `obj[i:j]` | `__getitem__(key)` | Subscriptable object |
| `with obj: ...` | `__enter__` / `__exit__` | Context manager |
| `for i in obj: ...` | `__iter__ / __next__` <br/>`__len__` / `__getitem__` | Iterable object <br/>Sequence|
| `obj.<attribute>` | `__getattr__` | Dynamic attribute retrieval |
| `obj(*args, **kwargs)` | `__call__(*args, **kwargs)` | Callable object |


In [12]:
# Anti-pattern - Mutable default arguments

def wrong_user_display(user_metadata: dict = {"name": "John", "age": 30}):
    name = user_metadata.pop("name")
    age = user_metadata.pop("age")
    return f"{name} ({age})"


print(wrong_user_display())
print(wrong_user_display({"name": "Jane", "age": 25}))
# print(wrong_user_display()) # KeyError: 'name'

# Right way
def user_display(user_metadata: dict = None):
    user_metadata = user_metadata or {"name": "John", "age": 30}
    name = user_metadata.pop("name")
    age = user_metadata.pop("age")
    return f"{name} ({age})"

print(user_display())
print(user_display({"name": "Jane", "age": 25}))
print(user_display())


John (30)
Jane (25)
John (30)
Jane (25)
John (30)


In [13]:
# Anti-pattern - Extending built-in types

"""
TIP: Don't extend directly from dict, use collections.UserDict instead. For
lists, use collections.UserList, and for strings,
use collections.UserString.
"""

class BadList(list):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "even"
        else:
            prefix = "odd"
        return f"[{prefix}] {value}"

bl = BadList((0, 1, 2, 3, 4, 5))
print(bl[0])
print(bl[1])

# "".join(bl) # TypeError: sequence item 0: expected str instance, int found


# Right way
from collections import UserList
class GoodList(UserList):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "even"
        else:
            prefix = "odd"
        return f"[{prefix}] {value}"

gl = GoodList((0, 1, 2))
print(gl[0]) # '[even] 0'
print(gl[1]) # '[odd] 1'
print("; ".join(gl)) # '[even] 0; [odd] 1; [even] 2'



[even] 0
[odd] 1
[even] 0
[odd] 1
[even] 0; [odd] 1; [even] 2
